In [1]:
# VER TODO-LIST em README.md
import json
import urllib.request
from toolset import (get_examples, get_frequency,
                    get_text, get_tokens)
from cltk import NLP
from cltk.lemmatize.grc import GreekBackoffLemmatizer
lemmatizer = GreekBackoffLemmatizer()

from cltk.alphabet.text_normalization import cltk_normalize

In [2]:
creed, wf = get_text("creed.txt")
tokens, token_set, phrases = get_tokens(creed)
frequency = get_frequency(tokens)
cltk_nlp = NLP(language="grc")
cltk_doc = cltk_nlp.analyze(creed)

wordlist = []
for pair in frequency:
    token = pair[0]
    wordlist.append(token)

‎𐤀 CLTK version '1.0.25'.
Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`, `GreekNERProcess`.


In [3]:
def request(action, **params):
    return {'action': action,
            'params': params,
            'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))

    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')

    if 'error' not in response:
        raise Exception('response is missing required error field')
    
    if 'result' not in response:
        raise Exception('response is missing required result field')

    if response['error'] is not None:
        raise Exception(response['error'])

    return response['result']

noteIDs = invoke('findNotes', query='deck:Liturgia')
notesInfo = invoke('notesInfo', notes=noteIDs)

In [4]:
notes = []
for card in notesInfo:
    notes.append(cltk_normalize(card['fields']['Word']['value']))
fwl = [cltk_normalize(word) for word in wordlist if word not in notes]
notes_group = lemmatizer.lemmatize(notes)
fwl_group = lemmatizer.lemmatize(fwl)

In [5]:
notes_lemmas = set([tuple[1] for tuple in notes_group])
fwl_lemmas = set([tuple[1] for tuple in fwl_group])

In [50]:
filtered_fwl_lemmas = [word for word in fwl_lemmas if word not in notes_lemmas]
reworked_fwl = [tuple for tuple in fwl_group if tuple[1] in filtered_fwl_lemmas]

In [7]:
'''
Ok, o que fazer agora, então?

O que eu tenho que fazer é o seguinte:
1. pegar as palavras que estão dispostas acima
2. Procurar por elas em dois sites:
2.1. BibleHub
2.2. Wiktionary 
3. Se nada for encontrado, procurar pelos seus *lemas*
4. Se nada for encontrado, criar uma lista para procurar manualmente.
'''

'\nOk, o que fazer agora, então?\n\nO que eu tenho que fazer é o seguinte:\n1. pegar as palavras que estão dispostas acima\n2. Procurar por elas em dois sites:\n2.1. BibleHub\n2.2. Wiktionary \n3. Se nada for encontrado, procurar pelos seus *lemas*\n4. Se nada for encontrado, criar uma lista para procurar manualmente.\n'

In [161]:
from words import Word as WikWord
from scraper import Word as BibleWord
from scraper import get_link, get_entry_soup, get_word_data, fetch_group_as_string
from scraper import BASE_URL

In [9]:
# antes é necessário fazer com que as palavras idênticas aos seus 
# lemas não provoquem redundância, desfazendo as tuplas
# em uma string única

In [213]:
for i in range(len(reworked_fwl)):
#    for j in reworked_fwl[i]:
    if reworked_fwl[i][0] == reworked_fwl[i][1]:
        reworked_fwl[i] = reworked_fwl[i][0]

In [11]:
working = reworked_fwl[0]
# wword = WikWord(working)

In [12]:
def fetch_bible_word(word):
    link = get_link(BASE_URL, word)
    soup = get_entry_soup(link)
    word = BibleWord(get_word_data(soup))

    return word

In [193]:
from requests.exceptions import MissingSchema
# this can be further optimized by instead of summing up the searches
# isolating the searches according to the site used to search


def double_searcher(word):
    if type(word) == tuple:
        try:
            bible_word = fetch_bible_word(word[0])
            return 1, 0, "Bible", word[0], bible_word.data

        except MissingSchema:
            try:
                bible_word = fetch_bible_word(word[1])
                return 2, 0, "Bible", word[0], bible_word.data

            except MissingSchema:
                wik_word = WikWord(word[0])
                if (not wik_word.definitions) or (wik_word.defintions == []):
                        wik_word = WikWord(word[1])
                        return 2, 2, "Wik", word[1], wik_word
                return 2, 1, "Wik", word[1], wik_word

    elif type(word) == str:
        try: 
            bible_word = fetch_bible_word(word)
            return 1, 0, "Bible", word, bible_word.data

        except:
            try:
                wik_word = WikWord(word)
                return 1, 1, "Wik", word, wik_word
            except:
                return 1, 1, False, word

    else:
        return 0, 0, 0, False

 

In [211]:
def searcher(word):
    if type(word) == tuple:
        try:
            bible_word = fetch_bible_word(word[0])
            return 1, "Bible", word[0], bible_word.data

        except MissingSchema:
            try:
                bible_word = fetch_bible_word(word[1])
                return 2, "Bible", word[0], bible_word.data

            except MissingSchema:

                return 2, False, word[0]
                

    elif type(word) == str:
        try: 
            bible_word = fetch_bible_word(word)
            return 1, "Bible", word, bible_word.data

        except:
                return 1, False, word

    else:
        return 0, False, word

In [222]:
reworked_fwl = reworked_fwl[29:]

In [223]:
reworked_fwl

[('ἐνανθρωπήσαντα', 'ἐνανθρωπέω'),
 'Σταυρωθέντα',
 ('ὑπὲρ', 'ὑπέρ'),
 'Ποντίου',
 'Πιλάτου',
 ('παθόντα', 'πάσχω'),
 ('ταφέντα', 'θάπτω'),
 ('ἀναστάντα', 'ἀνίστημι'),
 ('τρίτῃ', 'τρίτος'),
 'Γραφάς',
 ('ἀνελθόντα', 'ἀνέρχομαι'),
 'οὐρανοὺς',
 ('καθεζόμενον', 'κατάἕζομαι'),
 'πάλιν',
 ('μετὰ', 'μετά'),
 ('κρῖναι', 'κρίνω'),
 ('ζῶντας', 'ζάω'),
 ('νεκρούς', 'νεκρός'),
 ('βασιλείας', 'βασιλεία'),
 ('οὐκ', 'οὐ'),
 'τέλος',
 'Πνεῦμα',
 'Ἅγιον',
 ('ζωοποιόν', 'ζωοποιός'),
 ('ἐκπορευόμενον', 'ἐκπορεύω'),
 ('σὺν', 'σύν'),
 'Πατρὶ',
 'Υἱῷ',
 ('συμπροσκυνούμενον', 'σύν,πρόσκυνάω'),
 ('συνδοξαζόμενον', 'συνδοξάζω'),
 ('λαλῆσαν', 'λαλέω'),
 'Προφητῶν',
 'Εἰς',
 ('μίαν', 'εἷς'),
 ('ἁγίαν', 'ἅγιος'),
 'καθολικὴν',
 'ἀποστολικὴν',
 'Ἐκκλησίαν',
 'Ὁμολογῶ',
 ('ἓν', 'εἷς'),
 'βάπτισμα',
 ('ἄφεσιν', 'ἄφεσις'),
 'Προσδοκῶ',
 ('ἀνάστασιν', 'ἀνάστασις'),
 ('νεκρῶν', 'νεκρός'),
 ('μέλλοντος', 'μέλλω'),
 'Ἀμήν',
 ('ἕνα', 'εἷς'),
 'τε',
 ('πάντων', 'πᾶς'),
 'Θεοῦ',
 'Πατρὸς',
 ('γεννηθέντα', 'γεννάω'),
 'δι 

In [228]:
bible_searches = 0
wiki_searches = 0
data = []
blanks = []
import time

for index in range(len(reworked_fwl)):
    print("word no. {}: {}".format(index, reworked_fwl[index]))
    a = searcher(reworked_fwl[index])
    if a[1] != False:
        data.append((a[1], a[2], a[3]))
    else:
        blanks.append(a[2])
    bible_searches += a[0]
    if bible_searches >= 10:
        for n in range(60):
            time.sleep(1)
            print("Wait... {}/60".format(n))
            bible_searches = 0
"""
for index in range(len(reworked_fwl)):
    print("word no. {}".format(index))
    a = double_searcher(reworked_fwl[index])
    data.append((a[2], a[3], a[4]))
    bible_searches += a[0]
    wiki_searches += a[1]
    if bible_searches >= 10:
        for n in range(60):
            time.sleep(1)
            print("Wait... {}/60".format(n))
            bible_searches = 0
    if wiki_searches >= 10:
        for n in range(60):
            time.sleep(1)
            print("Wait... {}/60".format(n))
            wiki_searches = 0
"""

word no. 0: ('ἐνανθρωπήσαντα', 'ἐνανθρωπέω')
word no. 1: Σταυρωθέντα
word no. 2: ('ὑπὲρ', 'ὑπέρ')
word no. 3: Ποντίου
word no. 4: Πιλάτου
word no. 5: ('παθόντα', 'πάσχω')
word no. 6: ('ταφέντα', 'θάπτω')
Wait... 0/60
Wait... 1/60
Wait... 2/60
Wait... 3/60
Wait... 4/60
Wait... 5/60
Wait... 6/60
Wait... 7/60
Wait... 8/60
Wait... 9/60
Wait... 10/60
Wait... 11/60
Wait... 12/60
Wait... 13/60
Wait... 14/60
Wait... 15/60
Wait... 16/60
Wait... 17/60
Wait... 18/60
Wait... 19/60
Wait... 20/60
Wait... 21/60
Wait... 22/60
Wait... 23/60
Wait... 24/60
Wait... 25/60
Wait... 26/60
Wait... 27/60
Wait... 28/60
Wait... 29/60
Wait... 30/60
Wait... 31/60
Wait... 32/60
Wait... 33/60
Wait... 34/60
Wait... 35/60
Wait... 36/60
Wait... 37/60
Wait... 38/60
Wait... 39/60
Wait... 40/60
Wait... 41/60
Wait... 42/60
Wait... 43/60
Wait... 44/60
Wait... 45/60
Wait... 46/60
Wait... 47/60
Wait... 48/60
Wait... 49/60
Wait... 50/60
Wait... 51/60
Wait... 52/60
Wait... 53/60
Wait... 54/60
Wait... 55/60
Wait... 56/60
Wait... 

'\nfor index in range(len(reworked_fwl)):\n    print("word no. {}".format(index))\n    a = double_searcher(reworked_fwl[index])\n    data.append((a[2], a[3], a[4]))\n    bible_searches += a[0]\n    wiki_searches += a[1]\n    if bible_searches >= 10:\n        for n in range(60):\n            time.sleep(1)\n            print("Wait... {}/60".format(n))\n            bible_searches = 0\n    if wiki_searches >= 10:\n        for n in range(60):\n            time.sleep(1)\n            print("Wait... {}/60".format(n))\n            wiki_searches = 0\n'

In [227]:
searcher(reworked_fwl[0])

(2, False, 'ἐνανθρωπήσαντα')

In [253]:
werd = data[-4]
curr = werd[2]
curr['concordances']
b = get_context_example(werd[1], wf)
b

'τὸν μονογενῆ, τὸν ἐκ τοῦ Πατρὸς γεννηθέντα πρὸ πάντων τῶν αἰώνων. Φῶς ἐκ φω'

In [255]:
import pandas as pd
for item in data[-4:]:
    if item[0] == "Bible":

        print(item[1])

        curr = item[2]
        concordances = curr["concordances"]

        dc = {
            "word": concordances["Original Word"],
            "phonetics": concordances["Phonetic Spelling"],
            "category": concordances["Part of Speech"],
            "meaning": concordances['Definition'],
            "greek": fetch_group_as_string([tuple[0] for tuple in curr['examples']], single_list=True),
            "english": fetch_group_as_string([tuple[1] for tuple in curr['examples']], single_list=True),
        }

        dc["context"] = get_context_example(item[1], wf)
        
        # to display which word is being worked upon at the time

        words_dataframe = pd.DataFrame.from_dict(dc, orient="index")
        words_dataframe = words_dataframe.transpose()
        words_dataframe.to_csv("greek_output.csv", encoding="utf-8", mode="a", header=False, index=False)

γεννηθέντα
δι ̓


IndexError: list index out of range

In [89]:
import re
curr_d = data[0][1]
word_name = curr_d['concordances']['Original Word']
concordance = wf.concordance_list(word_name)[0][6]
cleaned_up = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', concordance)
pattern = r"^\s+"
cleaned_up = re.sub(pattern,"",cleaned_up)
cleaned_up

'Πιστεύω εἰς ἕνα Θεόν, Πατέρα, παντοκράτορ'

In [175]:
# ways to perfect this function:
# check whether the first word in the concordance example is it
# if not, cut off this word (so as to remove incomplete words)
# do the same to the last word of the sentence
def get_context_example(word, concordance):
    """
    for the purpose of getting an example of the word in question
    being used in context, and cleaning up this example 
    (because NLTK adds a lot of trailing spaces and separates words
    from punctuation signals in the string itself)
    """
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    context = concordance.concordance_list(word.split()[0])[0][6]
    cleaned_up_pre = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', context)
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    cleaned_up = re.sub(trailing_spaces_end, "", cleaned_up_start)

    return cleaned_up

def clean_up(string):
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    cleaned_up_pre = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', context)
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    cleaned_up = re.sub(trailing_spaces_end, "", cleaned_up_start)

    return cleaned_up

In [168]:
b = get_context_example(reworked_fwl[0][0], wf)

IndexError: list index out of range

In [189]:
wf.concordance_list(dc['word'].split()[0].split(",")[0])

[]

In [184]:
get_context_example(dc['word'].split()[0].split(",")[0],wf)

IndexError: list index out of range

In [192]:
# what I must get is not the word from bibleHub, but take that from it from BibleHub
# and then use the very word to look for examples in the concordance
data

[('Bible',
  {'concordances': {'Original Word': 'πιστεύω',
    'Part of Speech': 'Verb',
    'Transliteration': 'pisteuó',
    'Phonetic Spelling': "(pist-yoo'-o)",
    'Definition': 'to believe, entrust',
    'Usage': 'I believe, have faith in, trust in; pass: I am entrusted with.'},
   'examples': [('ὁ Ἰησοῦς Πιστεύετε ὅτι δύναμαι',
     ' Jesus Believe you that I am able'),
    ('τούτων τῶν πιστευόντων εἰς ἐμέ', ' of these who believe in me'),
    ('τῇ προσευχῇ πιστεύοντες λήμψεσθε ',
     ' prayer believing you will receive')]}),
 ('Bible',
  {'concordances': {'Original Word': 'θεός, οῦ, ὁ',
    'Part of Speech': 'Noun, Feminine; Noun, Masculine',
    'Transliteration': 'theos',
    'Phonetic Spelling': "(theh'-os)",
    'Definition': 'God, a god',
    'Usage': '(a) God, (b) a god, generally.'},
   'examples': [('δύναται ὁ θεὸς ἐκ τῶν', ' able is God from the'),
    ('πνεῦμα τοῦ θεοῦ καταβαῖνον ὡσεὶ', ' Spirit of God descending as'),
    ('εἶ τοῦ θεοῦ εἰπὲ ἵνα', ' you are of God sp

In [203]:
blank = []
for item in data:
    if item[0] != "Bible":
       blank.append(item) 

In [209]:
blank[1][2].name

'σαρκόω'

In [210]:
blank

[('Wik', 'ὁμοούσιος', <words.Word at 0x7f52f7b8bf60>),
 ('Wik', 'σαρκόω', <words.Word at 0x7f52f706ff28>)]